In [1]:
import re
import pandas as pd


from collections import defaultdict 

In [217]:
f1 = 'D:\\projects\\_external_files\\cod_files\\afg_adminboundaries_tabulardata.xlsx'
f2 = 'D:\\projects\\_external_files\\cod_files\\ukr_adminboundaries_tabulardata.xlsx'

files = [f1, f2]

In [243]:
def get_all_cols_for_level(df, lvl=0):

    return_df = pd.DataFrame(columns=['pcode','adm','location_name','adm_lvl'])

    while lvl >= 0:
        column_levels = [c for c in df.columns if str(lvl) in c]
        if len(column_levels) == 0:
            lvl -= 1
            continue
        
        
        pcodes = []
        names = []
        others = []
    
        for col in column_levels:
            pcode_match = re.match(r'(adm)(\d+)_pcode', col.lower())
            name_match = re.match(r'(adm)(\d+)_([a-z]{2,3})', col.lower())
            alt_match = re.match(r'(adm)(\d+)(alt)(\d+)_([a-z]{2,3})', col.lower())
            if pcode_match:
                pcodes.append(pcode_match.group(0))
            elif name_match:
                names.append(name_match.group(0))
            elif alt_match:
                #throw away alts for now
                pass
            else:
                others.append(col)
    
        #sanity checks
        if len(pcodes) != 1:
            print(f"unexpected condition - pcodes len should be 1. {pcodes}")
        if len(names) < 1:
            print(f"unexpected condition - names len should be at least 1. {names}")
        if len(others) > 0:
            print(f"unexpected condition, but not fatal - others len should be empty. {others}")

    
        required_columns = names
        required_columns.extend(pcodes)
        
    
        # Melting the DataFrame to combine columns into rows
        melted_df = df[required_columns].melt(id_vars=pcodes[0], var_name='adm', value_name='location_name').copy()
        melted_df['adm_lvl'] = lvl
        melted_df = melted_df.rename(columns={pcodes[0]:'pcode'})


        return_df = pd.concat([return_df, melted_df])

        lvl -= 1


    return_df['pcode_prefix'] = return_df['pcode'].apply(lambda x: x[0:2])
    
    return_df['lang_code'] = return_df['adm'].apply(lambda x: x.split('_')[1])
    country=''
    country = return_df['location_name'][(return_df['adm_lvl'] == 0) & (return_df['lang_code'] == 'en')].tolist()[0]
    return_df['country'] = country
    return_df = return_df.drop_duplicates().reset_index()
    return_df = return_df[return_df['location_name'].isna() == False]
    
    return return_df[['country','pcode_prefix','location_name','pcode','adm_lvl','lang_code']]



In [244]:
def standardize_column_names(df, case='lower'):
    cols = df.columns
    new_columns={}
    if case == 'lower':
        for c in cols:
            new_columns[c] = c.lower()
    elif case == 'upper':
        for c in cols:
            new_columns[c] = c.upper()

    return df.rename(columns=new_columns)
    


    

In [245]:
def process_cods(f):
    preferred_level = 'ADM3'
    backup_level = 'ADM2'

    # Create an ExcelFile object
    xls = pd.ExcelFile(f)
    sheet_names = xls.sheet_names

    if preferred_level in sheet_names:
        df = pd.read_excel(xls, sheet_name=preferred_level)
    else:
        df = pd.read_excel(xls, sheet_name=backup_level)

    


    return df


        

In [246]:
df_location = pd.DataFrame(columns=['country','pcode_prefix','location_name','pcode','adm_lvl','lang_code'])
for f in files:
    print(f)
    df = process_cods(f)
    df = standardize_column_names(df)
    df_new_loc = get_all_cols_for_level(df, lvl=4)


    df_location = pd.concat([df_location, df_new_loc])


    print(df_location.shape)

df_location


D:\projects\_external_files\cod_files\afg_adminboundaries_tabulardata.xlsx
(872, 6)
D:\projects\_external_files\cod_files\ukr_adminboundaries_tabulardata.xlsx
(6680, 6)


,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code
0,Afghanistan,AF,Khash,AF1705,2,en
1,Afghanistan,AF,Hazrat-e-Sultan,AF2002,2,en
2,Afghanistan,AF,Pul-e-Alam,AF0501,2,en
3,Afghanistan,AF,Mata Khan,AF1202,2,en
4,Afghanistan,AF,Gosfandi,AF2206,2,en
...,...,...,...,...,...,...
7572,Ukraine,UA,Киев,UA80,1,ru
7573,Ukraine,UA,Севастополь,UA85,1,ru
7574,Ukraine,UA,Ukraine,UA,0,en
7575,Ukraine,UA,Україна,UA,0,ua


In [247]:
df_location[df_location['pcode'] == 'UA0102009']

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code
2,Ukraine,UA,Verkhorichenska,UA0102009,3,en
1771,Ukraine,UA,Верхоріченська,UA0102009,3,ua
3540,Ukraine,UA,Верхореченский,UA0102009,3,ru


In [248]:
df_location.to_csv("c://temp//locations.csv", encoding='utf-8-sig', index=False)

# Now use the DF

In [263]:
def get_pcode_from_location(loc):
    return df_location['pcode'][df_location['location_name'] == loc].tolist()

print(get_pcode_from_location('Sharak-e-Hayratan'))
print(get_pcode_from_location('حضرت سلطان'))
print(get_pcode_from_location('Кальчикская'))



['AF2116']
['AF2002']
['UA1414001']


In [268]:
def get_adm_lvl_from_pcode(pcode):
    return list(set(df_location['adm_lvl'][df_location['pcode'] == pcode].tolist()))

print(get_adm_lvl_from_pcode('UA1414001'))

[3]


In [273]:
def get_name_in_lang(pcode, lang='en'):
    return list(set(df_location['location_name'][(df_location['pcode'] == pcode) & (df_location['lang_code'] == lang)].tolist()))

print(get_name_in_lang('UA1414001', 'en'))
print(get_name_in_lang('UA1414001', 'ru'))
print(get_name_in_lang('UA1414001', 'ua'))
print(get_name_in_lang('UA1414001', 'da'))

['Kalchytska']
['Кальчикская']
['Кальчицька']
[]


In [274]:
def crack_pcode_struct(df):
    #pass only one country in at a time
    countries = list(set(df['pcode_prefix'].tolist()))
    if len(countries) > 1:
        print("pass in only one country")
    else:
        pass

crack_pcode_struct(df_location)

pass in only one country


In [262]:
df_location

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code
0,Afghanistan,AF,Khash,AF1705,2,en
1,Afghanistan,AF,Hazrat-e-Sultan,AF2002,2,en
2,Afghanistan,AF,Pul-e-Alam,AF0501,2,en
3,Afghanistan,AF,Mata Khan,AF1202,2,en
4,Afghanistan,AF,Gosfandi,AF2206,2,en
...,...,...,...,...,...,...
7572,Ukraine,UA,Киев,UA80,1,ru
7573,Ukraine,UA,Севастополь,UA85,1,ru
7574,Ukraine,UA,Ukraine,UA,0,en
7575,Ukraine,UA,Україна,UA,0,ua


## END

In [38]:
def drop_junk_columns(df):
    # Counting NaN values in each column
    nan_count_per_column = df.isna().sum()
    
    # Get columns where NaN count exceeds 2
    columns_to_drop = nan_count_per_column[nan_count_per_column > 2].index
    
    # Drop columns
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

,ADM2_EN,ADM2_DA,ADM2_PCODE,ADM1_EN,ADM1_DA,ADM1_PCODE,ADM0_EN,ADM0_DA,ADM0_PCODE,DATE,VALIDON,REGION_EN,REGION_DA,REGIONCODE,UNITTYPE,AREA_SQKM
0,Khash,خاش,AF1705,Badakhshan,بدخشان,AF17,Afghanistan,افغانستان,AF,43081,44517,North Eastern,سهیل ختیځه حوزه,NE,District,242.907844
1,Hazrat-e-Sultan,حضرت سلطان,AF2002,Samangan,سمنگان,AF20,Afghanistan,افغانستان,AF,43081,44517,Northern,سهیلی حوزه,NR,District,2102.525804
2,Pul-e-Alam,پل علم,AF0501,Logar,لوگر,AF05,Afghanistan,افغانستان,AF,43081,44517,Capital,مرکزی حوزه,CR,Provincial Centre,1130.973208
3,Mata Khan,متاخان,AF1202,Paktika,پکتیکا,AF12,Afghanistan,افغانستان,AF,43081,44517,South Eastern,جنوب ختیځه حوزه,SE,District,405.445878
4,Gosfandi,گوسفندی,AF2206,Sar-e-Pul,سرپل,AF22,Afghanistan,افغانستان,AF,43081,44517,Northern,سهیلی حوزه,NR,District,620.344073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,Musa Khel,موسی خیل,AF1405,Khost,خوست,AF14,Afghanistan,افغانستان,AF,43081,44517,South Eastern,جنوب ختیځه حوزه,SE,District,470.240608
397,Dawlat Abad,دولت آباد,AF2909,Faryab,فاریاب,AF29,Afghanistan,افغانستان,AF,43081,44517,Northern,سهیلی حوزه,NR,District,2657.363420
398,Qara Bagh,قره باغ,AF0114,Kabul,کابل,AF01,Afghanistan,افغانستان,AF,43081,44517,Capital,مرکزی حوزه,CR,District,214.138621
399,Ab Kamari,آب کمری,AF3102,Badghis,بادغیس,AF31,Afghanistan,افغانستان,AF,43081,44517,Western,لویدیځه حوزه,WR,District,2311.046447
